In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


Kaggleメルカリチャレンジでは、販売者が投稿した情報を基に「適正な販売価格」を予測する
チャレンジです。訓練データとして、ユーザーが投稿した商品情報やカテゴリ、さらに商品の
状態やブランド名などが与えられており、それらを基に販売価格を予測するモデル作成が課題

In [ ]:
#必要なものインポート
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#jupyternotebookで画像表示のためインポート
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import train_test_split
#文字列内で %s などとし、% で続いて変換したい値を後ろに記述.format的なもの
#set_optionはpandasの浮動小数点桁数などの表示設定の設定値を変更する関数
pd.set_option('display.float_format',lambda x:'%.5f' % x)



In [ ]:
#tsvファイルはcsvファイルの兄弟で区切りがタブかカンマかの違い、設定すればcsv同様にexcelで表示できる
#tsvファイルもpd.read_csvで開くことができる
#今回のデータセットはtsv.7zとさらに圧縮ファイル化されてるからまずは解凍して通常のtsvファイルに直す

#データタイプを指定
types_dict_train = {'train_id':'int64', 'item_condition':'int8','price':'float64','shipping':'int8'}
types_dict_test = {'test_id':'int64', 'item_condition_id':'int8','shipping':'int8'}

#tsvファイルからPandasDataframeへ読み込み パス以降はtsvのための特殊な設定
train = pd.read_csv('../input/mercari-challenge/train.tsv', compression = 'infer',delimiter='\t', low_memory=True, dtype=types_dict_train)
test = pd.read_csv('../input/mercari-challenge/test.tsv',compression = 'infer', delimiter='\t', low_memory=True, dtype=types_dict_test)


In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
test.tail()

In [ ]:
print(train.shape, test.shape)

・train_id / test _id – ユーザー投稿のID
・name – 投稿タイトル。タイトルに価格に関する情報がある場合（例：$20）はメルカリが事前に削除し[rm]と置き換え。
・item_condition_id – ユーザーが指定した商品の状態
・category_name – 投稿カテゴリー
・brand_name – ブランドの名前
・price – 訓練データのみあり実際に売られた価格。米ドル表示。今回の予測ターゲット。
・shipping – 送料のフラグ。「1」は販売者負担。「0」は購入者負担。
・item_description – 商品説明の全文。タイトルと同様に価格情報あるなら[rm]と置き換え。

In [ ]:
#ファイルのデータに合わせてデータの統計量を確認するため関数つくる
#withはopen以外にもクラスなどそのブロック内で開いて閉じるという使い方できる
#withブロックでpd.option_context()を使うと、そのブロック内でのみ設定が変更
#pd設定したうえでincludeを使いよびだす
def display_all(df):
    with pd.option_context('display.max_rows',1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)
display_all(train.describe(include='all').transpose())

In [ ]:
'''
以下のカテゴリカルデータを数値変換
category_name
item_description
name
brand_name
'''
#'category'はpandasDF.dtype型の中の一つ
#後々catメソッドとか使う際に便利になる
train.category_name = train.category_name.astype('category')
train.item_description = train.item_description.astype('category')
train.name = train.name.astype('category')
train.brand_name = train.brand_name.astype('category')

test.category_name = test.category_name.astype('category')
test.item_description = test.item_description.astype('category')
test.name = test.name.astype('category')
test.brand_name = test.brand_name.astype('category')

#データ型を確認
train.dtypes, test.dtypes

In [ ]:
#重複の値のデータをみるためデータのユニーク値を確認
train.apply(lambda x: x.nunique())
test.apply(lambda x: x.nunique())

In [ ]:
#欠損値の数と各カラムで欠損が占める割合を確認 
train.isnull().sum(), train.isnull().sum()/train.shape[0]

In [ ]:
test.isnull().sum(), test.isnull().sum()/test.shape[0]

《データ前処理の流れ》
trainとtestのデータを連結させる
連結させたDataFrameの文字列のデータ形式を「cateogry」へ変換
文字列を数値へ値を変換
訓練用データの「price」をnp.log()で処理
ランダムフォレスト用にxとy（ターゲット）で分ける

ランダムフォレストのモデルを作るため、文字列データをPandas関数で一気に数値へと変換させる。
そのため、testとtrainで別々に処理を行わず、連結して事前処理を行う。

In [ ]:
#データ連結後でもどちらのデータに属すかわかるよう
#renameでtrainとtestのIDのカラム名を変更
train = train.rename(columns = {'train_id':'id'})
test = test.rename(columns = {'test_id':'id'})

#さらに、train,test別々に抜き出したりできるようにするため、識別するカラムを追加
#is_trainのみのカラム追加に注意
train['is_train'] = 1
test['is_train'] = 0

# trainのprice(価格）以外のデータをtestと連結
train_test_combine = pd.concat([train.drop(['price'],axis=1),test],axis=0)

train_test_combine.head()

In [ ]:
train_test_combine.shape

In [ ]:
#文字列のデータを数値へ変換
train_test_combine.category_name = train_test_combine.category_name.astype('category')
train_test_combine.item_description = train_test_combine.item_description.astype('category')
train_test_combine.name = train_test_combine.name.astype('category')
train_test_combine.brand_name = train_test_combine.brand_name.astype('category')

# combinedDataの文字列を「.cat.codes」で数値へ変換する
#catオブジェに様々なメソッドあるがcodeは数値へ変換でき、NANに対してはｰ1に変換
train_test_combine.name = train_test_combine.name.cat.codes
train_test_combine.category_name = train_test_combine.category_name.cat.codes
train_test_combine.brand_name = train_test_combine.brand_name.cat.codes
train_test_combine.item_description = train_test_combine.item_description.cat.codes

train_test_combine.dtypes

In [ ]:
train_test_combine.head()

In [ ]:
#testとtrainデータに分ける
df_test = train_test_combine.loc[train_test_combine['is_train']==0]
df_train = train_test_combine.loc[train_test_combine['is_train']==1]

df_test = df_test.drop(['is_train'], axis=1)
df_train = df_train.drop(['is_train'], axis=1)

df_test.shape, df_train.shape

In [ ]:
#df_trainへtrain['price']を戻す
df_train['price'] = train.price

# price（価格）をlog関数で処理
#対数をとる(対数変換)することで極端な数値による影響を低減させ、
#データの偏りが減り、より正規分布に近づく。
#説明変数の値が各データで差が大きいときによく使う
import numpy as np
df_train['price'] = df_train['price'].apply(lambda x: np.log(x) if x>0 else x)
df_train.head()

In [ ]:
#【ランダムフォレストでモデリング】
x_train, y_train = df_train.drop(['price'], axis=1), df_train.price
#n_jobsは並行処理するジョブの数
#None(1でデフォルト)と-1があり、-1は全てのプロセッサ活用
#min_sample_leafは葉ノードが必要とする最小サンプル数
#n_estimatorは森の中の木の数
m = RandomForestRegressor(n_jobs=-1, min_samples_leaf=5, n_estimators=200)
m.fit(x_train, y_train)
m.score(x_train, y_train)

In [ ]:
#上で出来上がったモデルを使い、テストデータを予測
pred = m.predict(df_test)
#np.exp(pred)で予測値を指数関数に変換
#numpyからpandasのseriesに変換
pred = pd.Series(np.exp(pred))
#テストデータのIDと予測値を結合
submit = pd.concat([df_test.id, pred], axis=1)
#メルカリに提出するためカラム名をメルカリコンペの指定に変更する
submit.columns = ['test_id', 'price']
submit.to_csv('submit_rf_base.csv', index=False)

In [ ]:
submit.to_csv('submission.csv', index=False)